In [ ]:
import cv2
import mediapipe as mp
import numpy as np

In [ ]:
def reference_from_wrist(results, hand):
    
    i = 0
    landmarks = []
    references = [0,0,0]
    for cords in results.multi_hand_landmarks[0].landmark:
        if i == 0:
            references[0] = cords.x
            references[1] = cords.y
            references[2] = cords.z
            landmarks.append([0,0,0])
    
        else:
            if hand == 'Right':
                landmarks.append([cords.x - references[0],cords.y - references[1],cords.z])
            else:
                landmarks.append([references[0] - cords.x,cords.y - references[1],cords.z])
        i += 1
    return landmarks

In [ ]:


#Rename to sign name
SIGNS = ['I','L','O','V','E','M','space','middle_finger']
CAPTURES = 50
image_count = CAPTURES
current = 0

X = []
y = []

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        annotated_image = image.copy()
        if results.multi_hand_landmarks:
            hand = results.multi_handedness[0].classification[0].label

            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        k = cv2.waitKey(1)
        title = 'Taking  points for image {0} {1}'.format(SIGNS[current],str(image_count))
        cv2.imshow("", annotated_image)
        cv2.setWindowTitle("", title)
        if k%256 == 32:
            if results.multi_hand_landmarks != None:
                #if space pressed
                new_landmarks = np.array(reference_from_wrist(results, hand))
                X.append(new_landmarks)
                y.append(SIGNS[current])

                cv2.destroyAllWindows()
                image_count -= 1
                if image_count == 0:
                    current += 1
                    image_count = CAPTURES
        if cv2.waitKey(5) & 0xFF == ord('q') or current == len(SIGNS):
            np.savez('landmarks/X', np.array(X))
            np.savez('landmarks/y', np.array(y))
            break
cap.release()
cv2.destroyAllWindows()